# Augmented Sketch Classification

Gabriel Ryan (gr2547)

Abhijit Suprem (asf2182)

## Introduction:

Sketch classification is an inherently difficult problem due the lack of information in most sketches and inconsistencies in how objects are drawn. Unlike images, sketches usually don't include information about textures and others details, instead only having an outline of the object. In addition people will use varying levels of abstraction in their sketches, resulting in sketches that vary from fairly realistic representations to completely symbolic abstractions. This variation can be seen in the following examples of sketches of monkeys:
<table>
<tr>
    <td> <img src="https://s3.amazonaws.com/gryan-content/dlcv_report/monkey2.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="https://s3.amazonaws.com/gryan-content/dlcv_report/monkey1.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="https://s3.amazonaws.com/gryan-content/dlcv_report/monkey3.png" alt="Drawing" style="width: 250px;"/> </td>
    <td> <img src="https://s3.amazonaws.com/gryan-content/dlcv_report/monkey4.png" alt="Drawing" style="width: 250px;"/> </td>
</tr>
</table>

While the first sketch seems fairly realistic, the following three sketches are increasingly abstract. However, these sketches still contain information in markings and shape of the face, belly, and tail that make them recognizable as monkeys in the middle two sketches. This suggests that when our brains perform classification and recognition, they are able to learn those abstractions from real life images of monkeys.

This project replicates that process by using processed real life images to improve classification of sketches. Our hypothesis is that networks could learn to identify and pick out the same patterns that our brains do when recognizing an image, and thus be able to learn classify sketches even if that sketch does not directly resemble sketches used in training.

The most widely used dataset for sketch classification is the TU-Berlin sketch dataset [1], which consists of 250 classes with 80 samples per class. The current best performance on it uses an ensemble of 5 networks that classify each step of the sketch and achieves an accuracy of 77.6% [2].

We demonstrate that preprocessing images to resemble sketches and using them as additional training data for classification on the TU-Berlin dataset improves performance. Specifically, for the a 3 class sample of the dataset, we were able to go from 85.0 to 90.1% by incorporating processed images into the training data. Unfortunately, the image processing was time consuming and we were unable to process enough images to test more classes, however, we believe the substantial improvement of performance on this sample is a convincing proof of concept.


## Sketchification:

In order to make images look like sketches, we trained a small network to perform binary classification of sketches using the Berkeley Segmentation Dataset and Benchmark [3]. 

## Preprocessing:
- explain filtering process and reasoning, demo on image from edge detection

## Proof of concept test:

### Results

### Analysis
- What were

## Testing on full network:

### Analysis

## Conclusions

## Attribution:

Per pixel sketchification of images was developed primarily by Abhijit Suprem. I (Gabriel Ryan) primarily developed the image preprocessing and sketch recognition networks.

## References

1. asdf
2. asdf
3. Martin, David R. et al. “A Database of Human Segmented Natural Images and its Application to Evaluating Segmentation Algorithms and Measuring Ecological Statistics.” ICCV (2001).